In [1]:
!pip install requests beautifulsoup4 tiktoken huggingface_hub
!pip install datasets

Importing necessary packages

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import tiktoken
from tqdm import tqdm
import pandas as pd

HTML Cleaning

In [3]:
"""
def clean_text(html_text):
    ""Odstraní HTML tagy a jiné nadbytečné znaky z textu.""
    soup = BeautifulSoup(html_text, "html.parser")
    text = soup.get_text()
    text = re.sub(r'\s+', ' ', text)  # Odstranění přebytečných bílých znaků
    return text.strip()
    
"""    
import re
from bs4 import BeautifulSoup
import html

def clean_text(html_text):
    """Odstraní HTML tagy, dekóduje HTML entity a odstraňuje skripty a styly."""
    # Parse HTML
    soup = BeautifulSoup(html_text, "html.parser")
    
    # Remove script and style tags
    for script_or_style in soup(["script", "style"]):
        script_or_style.decompose()
    
    # Get text
    text = soup.get_text(separator=" ")  # Přidá mezery mezi bloky
    
    # Decode HTML entities
    text = html.unescape(text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()



Text division

In [4]:
def split_text(text, max_length=10):  # max_length je nastaven na přibližný počet slov
    words = text.split()  # rozdělí na slova
    chunks = [" ".join(words[i:i+max_length]) for i in range(0, len(words), max_length)]
    return chunks

Function for webscraping

In [13]:
def scrape_multiple_pages(urls, num_pages=10):
    """Prochází URL a pro stránkovatelné URL iteruje přes více stránek."""
    texts = []
    for url in tqdm(urls): # tqdm vypisuje progres přes url a můžeme sledovat případné chyby
        for page in range(1, num_pages + 1):
            if '{page}' in url:  # Kontrola, zda URL obsahuje placeholder pro stránkování
                formatted_url = url.format(page=page)
            else:
                formatted_url = url  # Pro statické URL bez stránkování

            try:
                response = requests.get(formatted_url)
                if response.status_code == 200:
                    cleaned_text = clean_text(response.text)
                    chunks = split_text(cleaned_text)  # Rozdělit vyčištěný text do kratších částí
                    texts.extend(chunks)  # Přidat jednotlivé části do seznamu
                else:
                    print(f"Failed to retrieve {formatted_url}")
                    break
            except Exception as e:
                print(f"Error retrieving {formatted_url}: {e}")
                break
    return texts

In [8]:
for url in tqdm([ "https://zdopravy.cz/category/zeleznice/"]):
    for page in range(1, 10 + 1):
        if '{page}' in url:  # Kontrola, zda URL obsahuje placeholder pro stránkování
            formatted_url = url.format(page=page)
        else:
            formatted_url = url  # Pro statické URL bez stránkování
        
        print(formatted_url)

100%|██████████| 1/1 [00:00<00:00, 11366.68it/s]

https://zdopravy.cz/category/zeleznice/
https://zdopravy.cz/category/zeleznice/
https://zdopravy.cz/category/zeleznice/
https://zdopravy.cz/category/zeleznice/
https://zdopravy.cz/category/zeleznice/
https://zdopravy.cz/category/zeleznice/
https://zdopravy.cz/category/zeleznice/
https://zdopravy.cz/category/zeleznice/
https://zdopravy.cz/category/zeleznice/
https://zdopravy.cz/category/zeleznice/


Our chosen webpages - we are interested in traffic pages

In [14]:
urls = [
    "https://zdopravy.cz/category/zeleznice/",
    "https://zdopravy.cz/category/silnice/",
    "https://zdopravy.cz/category/letectvi/",
    "https://zdopravy.cz/category/voda/",
    "https://zdopravy.cz/category/mesto/",
    "https://cs.wikipedia.org/wiki/Parn%C3%AD_lokomotiva",
    "https://cs.wikipedia.org/wiki/Elektrick%C3%A1_jednotka",
    "https://cs.wikipedia.org/wiki/Dieselov%C3%A1_lokomotiva",
    "https://cs.wikipedia.org/wiki/Lokomotiva"
]

Let us now run webscraping...

In [15]:
# Spuštění webscrapingu pro zadaný počet stránek na každém zdroji
texts = scrape_multiple_pages(urls, num_pages=20)

100%|██████████| 9/9 [00:50<00:00,  5.56s/it]


... and count the number of tokens:

In [16]:
encoder = tiktoken.get_encoding("gpt2")
total_tokens = sum(len(encoder.encode(text)) for text in texts)

print(f"Celkový počet tokenů: {total_tokens}")
print(f"Počet textových bodů: {len(texts)}")

Celkový počet tokenů: 678520
Počet textových bodů: 19200


Save the scraped text as .csv file

In [18]:
data = pd.DataFrame(texts, columns=["text"])
#data.to_csv("zeleznice_dataset.csv", index=False, escapechar="\\")
data.to_csv("zeleznice_dataset.csv", index=False, escapechar="\\", encoding="utf-8-sig")